## Packages

In the last section we saw how to store useful code in a module and then use that functionality in a separate python script.

Sometimes you want to group several modules together in a logical unit: Python provides that capability with packages. Think of a package as a single folder where you can package any number of related modules together. 

For this example, we've written two modules: `daily` and `weekly` that return daily or weekly forecasts. We placed these modules in a folder called `weather` in the same directory as this notebook.

In [1]:
# daily.py
# 
# Provides the weather for Berkeley, CA for today

from urllib.request import urlopen
import json

def forecast():
    """
    Returns the daily weather for Berkeley, CA
    """
    response = urlopen('http://api.openweathermap.org/data/2.5/forecast/daily?q=Berkeley&mode=json&units=Imperial&cnt=1')
    rawWeatherData = response.read().decode("utf-8")
    weatherData = json.loads(rawWeatherData)

    forecastStr = "Forecast for Berkeley, CA: " + weatherData["list"][0]["weather"][0]["main"] + "\n" \
        "Current Temp: " + str(weatherData["list"][0]["temp"]["day"]) + " degrees \n"  \
        "High Temp: " + str(weatherData["list"][0]["temp"]["max"]) + " degrees \n" \
        "Low Temp: " + str(weatherData["list"][0]["temp"]["min"]) + " degrees"

    return forecastStr

In [2]:
# weekly.py
# 
# Provides the weather for Berkeley, CA for the week

from urllib.request import urlopen
import json
import datetime

def forecast():
    """
    Returns the daily weather for Berkeley, CA
    """
    response = urlopen('http://api.openweathermap.org/data/2.5/forecast/daily?q=Berkeley&mode=json&units=Imperial&cnt=7')
    rawWeatherDataList = response.read().decode("utf-8")
    weatherDataList = json.loads(rawWeatherDataList)

    forecastStr = ""
    for i in range(7):
        forecastStr += _daily_forecast(weatherDataList["list"][i]) + "\n\n"

    forecastStr = forecastStr[:-2] # Remove the two newlines at the end 
    return forecastStr


def _daily_forecast(weatherData):
    """
    Helper function that prints a single day's forecast
    """

    # Using python datetime support to convert a timestamp into a full date
    
    # First need to define the UTC offset for Berkeley, CA (UTC - 8:00) (not daylight savings time)
    current_utc_offset = -datetime.timedelta(hours=8)

    # Next we create a timezone based on the utc offset for Pacific Standard Time
    current_timezone = datetime.timezone(current_utc_offset)

    # Last we create a datetime object based on the timestamp provided by the response, and
    # we localize the timezone to represent Pacific Standard Time 
    current_datetime = datetime.datetime.fromtimestamp(weatherData["dt"], current_timezone)

    # Printing of the forecast
    
    # Note we use strftime to format how we would like to print out the datetime
    forecastStr = "Forecast for Berkeley, CA on " + current_datetime.strftime("%A, %B %d, %Y %H:%M %p") + " local time\n" \
    "Weather: "  + weatherData["weather"][0]["main"] + "\n" \
    "Current Temp: " + str(weatherData["temp"]["day"]) + " degrees \n"  \
    "High Temp: " + str(weatherData["temp"]["max"]) + " degrees \n" \
    "Low Temp: " + str(weatherData["temp"]["min"]) + " degrees"

    return forecastStr

The weather/daily.py module contains the ability to return the forecast at Berkeley, CA today. The weather/weekly.py module contains the ability to return the weekly forecast at Berkeley, CA. We moved those modules into a folder `weather`.  As a last step, we need to add an empty file named `__init.py__` to the weather folder. Once we create that empty file, we made `weather` into a package: a container that can contain modules and other packages.  We can import it into this notebook as follows:

In [3]:
from weather import daily, weekly

print("Daily:")
print(daily.forecast())
print()

print("Weekly:")
print(weekly.forecast())

Daily:
Forecast for Berkeley, CA: Clear
Current Temp: 79.5 degrees 
High Temp: 81.12 degrees 
Low Temp: 53.64 degrees

Weekly:
Forecast for Berkeley, CA on Saturday, August 22, 2015 12:00 PM local time
Weather: Clear
Current Temp: 79.5 degrees 
High Temp: 81.12 degrees 
Low Temp: 53.64 degrees

Forecast for Berkeley, CA on Sunday, August 23, 2015 12:00 PM local time
Weather: Clear
Current Temp: 81.21 degrees 
High Temp: 82.47 degrees 
Low Temp: 55.42 degrees

Forecast for Berkeley, CA on Monday, August 24, 2015 12:00 PM local time
Weather: Clear
Current Temp: 82.62 degrees 
High Temp: 85.41 degrees 
Low Temp: 56.8 degrees

Forecast for Berkeley, CA on Tuesday, August 25, 2015 12:00 PM local time
Weather: Clear
Current Temp: 86.61 degrees 
High Temp: 88.48 degrees 
Low Temp: 57.87 degrees

Forecast for Berkeley, CA on Wednesday, August 26, 2015 12:00 PM local time
Weather: Rain
Current Temp: 74.66 degrees 
High Temp: 84.61 degrees 
Low Temp: 52.2 degrees

Forecast for Berkeley, CA on Th

Note we could accomplish the same thing by importing only what we want from each module, but we will need to create an alias for each method since both the daily and weekly modules implement a module with the same name (`forecast`):

In [4]:
from weather.daily import forecast as daily_forecast
from weather.weekly import forecast as weekly_forecast

print("Daily:")
print(daily_forecast())
print("\n")

print("Weekly:")
print(weekly_forecast())

Daily:
Forecast for Berkeley, CA: Clear
Current Temp: 79.5 degrees 
High Temp: 81.12 degrees 
Low Temp: 53.64 degrees


Weekly:
Forecast for Berkeley, CA on Saturday, August 22, 2015 12:00 PM local time
Weather: Clear
Current Temp: 79.5 degrees 
High Temp: 81.12 degrees 
Low Temp: 53.64 degrees

Forecast for Berkeley, CA on Sunday, August 23, 2015 12:00 PM local time
Weather: Clear
Current Temp: 81.21 degrees 
High Temp: 82.47 degrees 
Low Temp: 55.42 degrees

Forecast for Berkeley, CA on Monday, August 24, 2015 12:00 PM local time
Weather: Clear
Current Temp: 82.62 degrees 
High Temp: 85.41 degrees 
Low Temp: 56.8 degrees

Forecast for Berkeley, CA on Tuesday, August 25, 2015 12:00 PM local time
Weather: Clear
Current Temp: 86.61 degrees 
High Temp: 88.48 degrees 
Low Temp: 57.87 degrees

Forecast for Berkeley, CA on Wednesday, August 26, 2015 12:00 PM local time
Weather: Rain
Current Temp: 74.66 degrees 
High Temp: 84.61 degrees 
Low Temp: 52.2 degrees

Forecast for Berkeley, CA on T

##Note on datetimes in Python 3

You may have noticed that the weather/weekly module had a number of lines of code to handle the date and time for the forecast. Time may be a simple concept for humans to understand, but there are so many different nuances that need to be explicitly built out in code. Here are a few of the nuances that need to be handled:

1. Time zones: the same point in time can be a different date somewhere else in the world
2. Calendars: there are different calendars other than the Gregorian calendar that we use
3. Formatting time: how do you want to display the date and time? for example 8/21/15 3 pm, or 2015-08-15 15:00?
4. Time deltas: If you add two days to August 30th, the object needs to be smart enough to know that the new date is September 1st and not August 32nd

We're are introducing the core datetime object in Python 3 that will give you the ability to manipulate datetimes in python which will prove very useful in data mining. Let's focus on the pieces of code in the weather/weekly module that handle these issues:

In [5]:
# Adding this line to ensure that this code works outside its context:
import datetime
weatherData = {}
weatherData["dt"] = 1440270976

# First need to define the UTC offset for Berkeley, CA (UTC - 8:00)
current_utc_offset = -datetime.timedelta(hours=8)

# Next we create a timezone based on the utc offset for Pacific Standard Time (not daylight savings time)
current_timezone = datetime.timezone(current_utc_offset)

# Last we create a datetime object based on the timestamp provided by the response, and
# we localize the timezone to make it "aware"
current_datetime = datetime.datetime.fromtimestamp(weatherData["dt"], current_timezone)

# Printing of the forecast

# Note we use strftime to format how we would like to print out the datetime
forecastStr = "Forecast for Berkeley, CA on " + current_datetime.strftime("%A, %B %d, %Y %H:%M %p") + " local time\n" \

# printing result for demonstration purposes
print(forecastStr)

Forecast for Berkeley, CA on Saturday, August 22, 2015 11:16 AM local time



The api response to openweathermap.org gives us a timestamp like this `"1440270976"`. This number is the number of seconds after "The Unix Epoch" the current time is. The "Unix Epoch" is January 1st, 1970 and you can [read all about it here.](https://en.wikipedia.org/wiki/Unix_time#History)

Our goal is to convert this string into a date time object that we can use for various things, most noteably print out a formatted string like this: "Saturday, August 22, 2015 11:16 AM local time".

First we want to define the timezone that the time currently represents. If we do not do this, then the python script will use the local time zone, so if someone in India would run this script they will see that current time in their timezone, which could be off by as much as a day!

For this example we set the offset to be Pacific Standard Time (not daylight savings time) by the following line:

`current_utc_offset = -datetime.timedelta(hours=8)`

The datetime.timedelta class defines a difference in times so that we can properly add and subtract time from date and times (this avoids the adding two days to August 30th to be August 32nd problem).

Next we create a timezone based on that offset from UTC: 

`current_timezone = datetime.timezone(current_utc_offset)`

The datetime.timezone class defines timezones, however this class cannot handle nuances such as daylight savings time, so depending on when you run this script it may be off by an hour if Berkeley is currently observing daylight savings time. We highly suggest using packages such as [pytz](https://pypi.python.org/pypi/pytz/) which enables you to look up timezone objects using descriptive names, like "US/Eastern").

Now that we have our timezone and our timestamp, we are ready to create our datetime object:

`current_datetime = datetime.datetime.fromtimestamp(weatherData["dt"], current_timezone)`

We use the datetime.datetime.frontimestamp function to create the datetime object using our timestamp and the Pacific Standard Time timezone object we created earlier.

Now that we have our datetime object, we can do a number of powerful operations with it such as subtract and add time, get the time in different timezones, and do comparisons with other datetime objects. Check out the [Python Standard Library section on datetimes](https://docs.python.org/3/library/datetime.html) for more information.

For this example we used the datetime object to print a nicely formatted time for each of the days we are forecasting:

`current_datetime.strftime("%A, %B %d, %Y %H:%M %p")`

The strftime function enables you to print a datetime in [various different formats](https://docs.python.org/3/library/datetime.html#strftime-strptime-behavior). We used the options so that we can print out a natural looking date time.